In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [ ]:
!pip install wandb


In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ziprikk (ziprikk-university-duisburg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
#!pip uninstall pandas torch-scatter torch-sparse torch-cluster torch-spline-conv optuna torch-geometric deeprobust numpy -y

In [ ]:
!pip install "pandas==2.2.2" "numpy<2.0,>=1.18.5"

In [ ]:

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install optuna
!pip install torch-geometric
!pip install deeprobust

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html


In [ ]:
!pip install optuna
!pip install torch-geometric
!pip install deeprobust

In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.data import DataLoader

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")


Torch version: 2.0.0+cu117
CUDA available: True
PyTorch Geometric version: 2.6.1


In [ ]:
# Standard library imports
import json
import logging
import math
import os
import gc
from contextlib import nullcontext
from copy import deepcopy
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import concurrent.futures
import io
import pickle
import warnings
import zlib
from wandb.sdk.wandb_run import Run
# Third-party imports
import numpy as np
import pandas as pd
import optuna
import psutil
import wandb
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    confusion_matrix,
    f1_score,
    matthews_corrcoef,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import RobustScaler, StandardScaler
from tqdm import tqdm

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch.cuda.amp import GradScaler, autocast
from torch.distributed import destroy_process_group, init_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim.lr_scheduler import LinearLR, OneCycleLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, WeightedRandomSampler

from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import MessagePassing, SAGEConv
from torch_geometric.transforms import BaseTransform
from torch_geometric.utils import to_networkx

import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from sklearn.metrics import (
    f1_score, roc_auc_score, precision_recall_curve, auc,
    average_precision_score, confusion_matrix, jaccard_score,
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
)
import torch
import numpy as np
from torch_geometric.loader import NeighborLoader
from sklearn.utils import compute_class_weight

import copy
import deeprobust.graph.utils as utils
from google.colab import drive
from sklearn.metrics import recall_score
import warnings
from typing import Optional
import numpy as np
from sklearn.metrics import (
    precision_recall_curve, auc, jaccard_score,
    confusion_matrix, matthews_corrcoef, cohen_kappa_score,
    balanced_accuracy_score, average_precision_score
)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
warnings.filterwarnings('ignore', message='.*pyg-lib.*')


In [ ]:
class EnhancedModelTrainer:

    def __init__(self, checkpoint_dir: str, wandb_project: str,
                    wandb_entity: str, wandb_run=None, oversample_ratio=50.0):

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(f"\nUsing device: {self.device}")

        if torch.cuda.is_available():

            print(f"Initial GPU memory allocated: {torch.cuda.memory_allocated(0) / 1e6:.2f} MB")
            print(f"Initial GPU memory cached: {torch.cuda.memory_reserved(0) / 1e6:.2f} MB")

        self.data_handler = DataHandler(checkpoint_dir, device=self.device)

        raw_edges, raw_features = self.data_handler.load_preprocessed_data()

        self.raw_features = raw_features.copy()

        self.df_features, self.df_edges = DataHandler.preprocess_with_smote(
            raw_features, raw_edges,
            oversample_ratio=oversample_ratio,
            start_timestep=1,
            end_timestep=36
        )

        self.wandb_project = wandb_project
        self.wandb_entity = wandb_entity
        self.wandb_run = wandb_run

        self.input_dim = len(self.df_features.columns.difference(
            ['node_id', 'Time step', 'class', 'binary_class']
        ))

        feature_cols = self.df_features.columns.difference(['node_id', 'Time step', 'class', 'binary_class'])
        print("\nFeature details:")
        print(f"Number of features: {len(feature_cols)}")
        print("\nFeature names:")
        for i, col in enumerate(feature_cols, 1):
            print(f"{i}. {col}")

        print("\nFeature statistics:")
        print(self.df_features[feature_cols].describe())

        # Print original class distribution
        print("\nOriginal class distribution:")
        print(self.df_features['class'].value_counts().sort_index())

        unique_classes = sorted(self.df_features['class'].unique())
        print(f"\nUnique classes found in data: {unique_classes}")

        binary_classes = [c for c in unique_classes if c != 3]
        print(f"Classes used for binary classification: {binary_classes}")

        if len(binary_classes) != 2:
            raise ValueError(f"Expected 2 classes (excluding class 3), found {len(binary_classes)}: {binary_classes}")


        self.class_mapping = {1: 1, 2: 0}
        self.df_features['binary_class'] = self.df_features['class'].map(lambda x: self.class_mapping.get(x, -1))
        self.raw_features['binary_class'] = self.raw_features['class'].map(lambda x: self.class_mapping.get(x, -1))

        print(f"\nClass mapping:")
        print(f"Class {binary_classes[0]} → 0")
        print(f"Class {binary_classes[1]} → 1")
        print(f"Class 3 → -1 (excluded from training)")

        binary_mask = self.df_features['class'].isin(binary_classes)
        self.train_mask = binary_mask & self.df_features['Time step'].between(1, 36)
        self.val_mask = binary_mask & self.df_features['Time step'].between(37, 41)
        self.test_mask = binary_mask & self.df_features['Time step'].between(42, 49)

        print("\nData split sizes (excluding class 3):")
        print(f"Training nodes: {self.train_mask.sum()}")
        print(f"Validation nodes: {self.val_mask.sum()}")
        print(f"Test nodes: {self.test_mask.sum()}")


        self._print_class_distribution_per_split()

        self.train_mask = torch.tensor(self.train_mask.values, dtype=torch.bool, device=self.device)
        self.val_mask = torch.tensor(self.val_mask.values, dtype=torch.bool, device=self.device)
        self.test_mask = torch.tensor(self.test_mask.values, dtype=torch.bool, device=self.device)

        raw_binary_mask = self.raw_features['class'].isin(binary_classes)
        raw_binary_classes_df = self.raw_features[raw_binary_mask]
        raw_class_counts = raw_binary_classes_df['binary_class'].value_counts()

        self.class_weights = torch.tensor(
            [1.0 / (raw_class_counts[i] / len(raw_binary_classes_df)) for i in range(2)],
            dtype=torch.float32,
            device=self.device
        )

        print("\nClass weights calculated on RAW data (before oversampling):")
        print(f"Class 0 weight: {self.class_weights[0]:.4f}")
        print(f"Class 1 weight: {self.class_weights[1]:.4f}")

        if self.wandb_run is not None:
            self.wandb_run.config.update({
                "binary_class_distribution": raw_class_counts.to_dict(),
                "class_3_nodes": len(self.raw_features[self.raw_features['class'] == 3]),
                "class_mapping": self.class_mapping,
                "train_nodes": self.train_mask.sum().item(),
                "val_nodes": self.val_mask.sum().item(),
                "test_nodes": self.test_mask.sum().item(),
                "oversample_timesteps": "1-36 only"  # Added to track this configuration
            })

        all_timesteps = sorted(self.df_features['Time step'].unique())
        self.train_timesteps = [ts for ts in all_timesteps if 1 <= ts <= 36]
        self.val_timesteps = [ts for ts in all_timesteps if 37 <= ts <= 41]
        self.test_timesteps = [ts for ts in all_timesteps if 42 <= ts <= 49]

    def extract_node_embeddings(self, model, timesteps=None, output_path=None):

        model.eval()
        data_list = []

        if timesteps is None:
            timesteps = sorted(self.df_features['Time step'].unique())

        if hasattr(model, 'num_neighbors'):
            num_neighbors_param = model.num_neighbors
        else:
            num_neighbors_param = [15]

        print(f"Using num_neighbors: {num_neighbors_param}")
        print("Model structure:")
        print(model)

        print("\nModel modules:")
        for name, module in model.named_modules():
            print(f"  {name}: {type(module).__name__}")

        with torch.no_grad():
            for timestep in timesteps:
                print(f"Processing timestep {timestep}")

                try:
                    data = self.data_handler.process_timestep_with_sampling(
                        self.df_edges,
                        self.df_features,
                        timestep,
                        num_neighbors=num_neighbors_param
                    ).to(self.device)
                    valid_mask = (data.y != -1)
                    if valid_mask.sum() == 0:
                        print(f"  No valid nodes for timestep {timestep}")
                        continue

                    embeddings_container = []

                    def input_hook_fn(module, input):

                        if isinstance(input, tuple) and len(input) > 0:
                            embeddings_container.append(input[0].detach())

                    def output_hook_fn(module, input, output):

                        embeddings_container.append(output.detach())

                    final_layer = None


                    for name, module in model.named_modules():
                        if isinstance(module, torch.nn.Linear):

                            if module.out_features == 2:
                                final_layer = module
                                print(f"Found final classification layer: {name}")
                                break

                    if final_layer:
                        hook = final_layer.register_forward_pre_hook(input_hook_fn)
                        print("Using pre-hook on classification layer to capture embeddings")
                    else:

                        for name, module in model.named_modules():
                            if any(conv_name in type(module).__name__.lower() for conv_name in
                                  ['conv', 'sage', 'gat', 'gcn']):
                                final_layer = module
                                print(f"Found graph convolution layer: {name}")
                                break

                        if final_layer:
                            # Hook the output of the graph conv layer
                            hook = final_layer.register_forward_hook(output_hook_fn)
                            print("Using hook on graph convolution layer to capture embeddings")
                        else:

                            for name, module in model.named_modules():
                                if isinstance(module, torch.nn.Module) and not isinstance(module, torch.nn.Sequential):
                                    if len(list(module.parameters())) > 0:
                                        final_layer = module
                                        print(f"Using fallback layer: {name}")
                                        break

                            if final_layer:
                                hook = final_layer.register_forward_hook(output_hook_fn)
                                print("Using fallback hook")
                            else:
                                raise ValueError("Could not find any suitable layer for hooking")

                    _ = model(data.x, data.edge_index)

                    hook.remove()

                    if not embeddings_container:
                        raise ValueError("No embeddings captured by hook")

                    node_embeddings = embeddings_container[0]
                    print(f"Captured embeddings shape: {node_embeddings.shape}")

                    embeddings_np = node_embeddings.cpu().numpy()

                    original_node_ids = self.df_features[self.df_features['Time step'] == timestep]['node_id'].values

                    if len(original_node_ids) != embeddings_np.shape[0]:
                        print(f"Warning: Number of node IDs ({len(original_node_ids)}) doesn't match number of embeddings ({embeddings_np.shape[0]})")

                        if len(original_node_ids) > embeddings_np.shape[0]:

                            timestep_mask = self.df_features['Time step'] == timestep
                            node_ids_mask = np.isin(self.df_features[timestep_mask]['node_id'].values, data.batch.cpu().numpy())

                            original_node_ids = self.df_features[timestep_mask]['node_id'].values[node_ids_mask]
                            print(f"Adjusted node IDs count: {len(original_node_ids)}")

                        min_length = min(len(original_node_ids), embeddings_np.shape[0])
                        original_node_ids = original_node_ids[:min_length]
                        embeddings_np = embeddings_np[:min_length]

                    # Get labels
                    labels = data.y.cpu().numpy()
                    if len(labels) != embeddings_np.shape[0]:
                        print(f"Warning: Number of labels ({len(labels)}) doesn't match number of embeddings ({embeddings_np.shape[0]})")
                        # Use the minimum to avoid errors
                        min_length = min(len(labels), embeddings_np.shape[0])
                        labels = labels[:min_length]
                        embeddings_np = embeddings_np[:min_length]

                    df_timestep = pd.DataFrame(
                        embeddings_np,
                        columns=[f'embedding_{i}' for i in range(embeddings_np.shape[1])]
                    )


                    df_timestep['node_id'] = original_node_ids
                    df_timestep['timestep'] = timestep
                    df_timestep['label'] = labels

                    data_list.append(df_timestep)

                except Exception as e:
                    print(f"Error processing timestep {timestep}: {str(e)}")
                    import traceback
                    traceback.print_exc()
                finally:

                    if 'data' in locals():
                        del data
                    if 'node_embeddings' in locals():
                        del node_embeddings
                    if 'embeddings_container' in locals():
                        del embeddings_container
                    torch.cuda.empty_cache()

        if not data_list:
            print("No embeddings extracted!")
            return None

        df_embeddings = pd.concat(data_list, ignore_index=True)
        if output_path:
            try:
                print(f"Saving embeddings to {output_path}")
                temp_path = "temp_node_embeddings.parquet"
                df_embeddings.to_parquet(temp_path)


                import shutil
                if output_path != temp_path:
                    shutil.move(temp_path, output_path)

                print(f"Saved embeddings successfully!")
                print(f"Total rows: {len(df_embeddings)}")
                print(f"Number of embedding dimensions: {df_embeddings.columns.str.startswith('embedding_').sum()}")
                print(f"Number of unique nodes: {df_embeddings['node_id'].nunique()}")
                print(f"Number of timesteps: {df_embeddings['timestep'].nunique()}")
            except Exception as e:
                print(f"Error saving file: {str(e)}")
                import os
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                    print("Cleaned up temporary file")

        return df_embeddings

    def get_embeddings(self, x, edge_index, edge_weight=None, model=None):

        if model is None:
            raise ValueError("Model must be provided to extract embeddings")

        x = F.dropout(x, p=model.dropout, training=model.training)

        for i, conv in enumerate(model.convs[:-1]):
            if edge_weight is not None:
                x_new = conv(x, edge_index, edge_weight)
            else:
                x_new = conv(x, edge_index)

            if hasattr(model, 'residual') and model.residual and x.size() == x_new.size():
                x = x_new + x
            else:
                x = x_new

            x = F.relu(x)
            x = F.dropout(x, p=model.dropout, training=model.training)

        return x


    def calculate_composite_score(self, y_true: np.ndarray, y_pred: np.ndarray, y_prob: np.ndarray) -> float:
        balanced_acc = balanced_accuracy_score(y_true, y_pred)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred != 1))
        fp = np.sum((y_true != 1) & (y_pred == 1))

        minority_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        minority_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        minority_f1 = (2 * minority_precision * minority_recall / (minority_precision + minority_recall)
                      if (minority_precision + minority_recall) > 0 else 0)
        composite_score = 0.1 * balanced_acc + 0.5 * minority_f1 + 0.4 * minority_recall
        return float(composite_score)

    def find_optimal_threshold(self, y_true: np.ndarray, y_prob: np.ndarray, step: float = 0.01) -> float:
        thresholds = np.arange(0.0, 1.0 + step, step)
        best_score = -np.inf
        best_threshold = 0.5
        for threshold in thresholds:
            y_pred = (y_prob >= threshold).astype(int)
            score = self.calculate_composite_score(y_true, y_pred, y_prob)
            if score > best_score:
                best_score = score
                best_threshold = threshold
        return best_threshold

    def _evaluate_on_test_sample(self, model, test_timesteps, num_neighbors):

        test_metrics = []

        for timestep in test_timesteps:
            print(f"Evaluating on test timestep {timestep}...")

            test_data = self.data_handler.process_timestep_with_sampling(
                self.df_edges,
                self.df_features,
                timestep,
                num_neighbors
            ).to(self.device)

            valid_mask = (test_data.y != -1)
            if valid_mask.sum() == 0:
                print(f"  No valid nodes for timestep {timestep}")
                continue

            model.eval()
            with torch.no_grad():
                out = model(test_data.x, test_data.edge_index)
                out_valid = out[valid_mask]
                probs = F.softmax(out_valid, dim=1)

                y_test = test_data.y[valid_mask].cpu().numpy()
                probs_np = probs.cpu().numpy()
                preds_np = (probs_np[:, 1] >= self.optimal_threshold).astype(int)

                if len(np.unique(y_test)) > 1:
                    metrics = self.calculate_metrics(y_test, preds_np, probs_np)
                    metrics['timestep'] = timestep
                    test_metrics.append(metrics)


                    print(f"  Composite Score: {metrics.get('composite_score', 0):.4f}")
                    print(f"  Class 1 Recall: {metrics.get('class_1_recall', 0):.4f}")
                    print(f"  Class 1 F1: {metrics.get('class_1_f1', 0):.4f}")
                else:
                    print(f"  Only one class present in timestep {timestep}, skipping metrics")

                del test_data, out, probs


        if test_metrics:
            avg_metrics = {
                k: np.mean([m.get(k, 0) for m in test_metrics if k in m])
                for k in test_metrics[0].keys()
                if k != 'timestep' and k != 'confusion_matrix'
            }
            return avg_metrics

        return {'composite_score': 0.0}


    def debug_mask_creation(self):
        print("\n=== Detailed Mask Class Distribution by Timestep ===")
        unique_classes = sorted(self.df_features['class'].unique())
        print(f"ALL Unique Classes in Dataset: {unique_classes}")

        timesteps = sorted(self.df_features['Time step'].unique())

        print(f"\n{'Timestep':^10} | {'Total Nodes':^12} | " +
              " | ".join([f"{'Class ' + str(cls):^10}" for cls in unique_classes]))
        print("-" * (50 + 12 * len(unique_classes)))

        for ts in timesteps:
            ts_data = self.df_features[self.df_features['Time step'] == ts]
            class_counts = ts_data['class'].value_counts()
            row_data = [
                ts,
                len(ts_data)
            ] + [class_counts.get(cls, 0) for cls in unique_classes]

            print(f"{row_data[0]:^10} | {row_data[1]:^12} | " +
                  " | ".join([f"{count:^10}" for count in row_data[2:]]))

    def _print_class_distribution_per_split(self):

        binary_mask = self.df_features['class'] != 3

        print("\n=== Class Distribution Per Split ===")


        train_data = self.df_features[binary_mask & self.df_features['Time step'].between(1, 36)]
        train_class_dist = train_data['binary_class'].value_counts().sort_index()
        print("\nTraining set class distribution:")
        print(train_class_dist)
        print(f"Class 0 percentage: {100 * train_class_dist[0] / train_class_dist.sum():.2f}%")
        print(f"Class 1 percentage: {100 * train_class_dist[1] / train_class_dist.sum():.2f}%")
        print(f"Class ratio (0:1): 1:{train_class_dist[1] / train_class_dist[0]:.2f}")


        val_data = self.df_features[binary_mask & self.df_features['Time step'].between(37, 41)]
        val_class_dist = val_data['binary_class'].value_counts().sort_index()
        print("\nValidation set class distribution:")
        print(val_class_dist)
        print(f"Class 0 percentage: {100 * val_class_dist[0] / val_class_dist.sum():.2f}%")
        print(f"Class 1 percentage: {100 * val_class_dist[1] / val_class_dist.sum():.2f}%")
        print(f"Class ratio (0:1): 1:{val_class_dist[1] / val_class_dist[0]:.2f}")


        test_data = self.df_features[binary_mask & self.df_features['Time step'].between(42, 49)]
        test_class_dist = test_data['binary_class'].value_counts().sort_index()
        print("\nTest set class distribution:")
        print(test_class_dist)
        print(f"Class 0 percentage: {100 * test_class_dist[0] / test_class_dist.sum():.2f}%")
        print(f"Class 1 percentage: {100 * test_class_dist[1] / test_class_dist.sum():.2f}%")
        print(f"Class ratio (0:1): 1:{test_class_dist[1] / test_class_dist[0]:.2f}")


    def calculate_metrics(self, targets: np.ndarray, predictions: np.ndarray, probabilities: np.ndarray) -> Dict:
        if len(targets) == 0 or len(predictions) == 0 or len(probabilities) == 0:
            return {}

        if predictions.ndim > 1 and predictions.shape[1] > 1:

            predictions = np.argmax(predictions, axis=1)

        if probabilities.ndim == 1:

            probabilities = np.vstack((1 - probabilities, probabilities)).T

        targets = targets.flatten()
        predictions = predictions.flatten()

        metrics = {}

        try:

            metrics['accuracy'] = (predictions == targets).mean()
            metrics['balanced_accuracy'] = balanced_accuracy_score(targets, predictions)

            unique_classes = np.unique(targets)
            if len(unique_classes) < 2:
                print(f"Warning: Only class {unique_classes[0]} present in targets. Some metrics will be unavailable.")
                return {'accuracy': metrics['accuracy']}


            metrics['precision'] = precision_score(targets, predictions)
            metrics['recall'] = recall_score(targets, predictions)


            metrics['f1'] = f1_score(targets, predictions, average=None)
            metrics['f1_micro'] = f1_score(targets, predictions, average='micro')
            metrics['f1_macro'] = f1_score(targets, predictions, average='macro')
            metrics['f1_weighted'] = f1_score(targets, predictions, average='weighted')

            metrics['auroc'] = roc_auc_score(targets, probabilities[:, 1])
            metrics['auprc'] = average_precision_score(targets, probabilities[:, 1])

            metrics['jaccard'] = jaccard_score(targets, predictions, average='weighted')
            metrics['mcc'] = matthews_corrcoef(targets, predictions)
            metrics['norm_mcc'] = (metrics['mcc'] + 1) / 2
            metrics['kappa'] = cohen_kappa_score(targets, predictions)


            metrics['confusion_matrix'] = confusion_matrix(targets, predictions)

            precision, recall, _ = precision_recall_curve(targets, probabilities[:, 1])
            metrics['pr_auc'] = auc(recall, precision)

            for class_idx in range(2):
                y_true_bin = (targets == class_idx)
                y_pred_bin = (predictions == class_idx)

                tp = np.logical_and(y_true_bin, y_pred_bin).sum()
                fp = np.logical_and(np.logical_not(y_true_bin), y_pred_bin).sum()
                fn = np.logical_and(y_true_bin, np.logical_not(y_pred_bin)).sum()

                precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0
                recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0

                metrics[f'class_{class_idx}_precision'] = precision_val
                metrics[f'class_{class_idx}_recall'] = recall_val
                metrics[f'class_{class_idx}_f1'] = (2 * precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
                metrics[f'class_{class_idx}_jaccard'] = jaccard_score(
                    targets == class_idx,
                    predictions == class_idx
                )

                metrics[f'class_{class_idx}_accuracy'] = np.mean(predictions[targets == class_idx] == class_idx) if np.sum(targets == class_idx) > 0 else 0

            metrics['composite_score'] = self.calculate_composite_score(targets, predictions, probabilities)

        except Exception as e:
            print(f"Error calculating metrics: {str(e)}")

            try:
                metrics = {
                    'accuracy': (predictions == targets).mean(),
                    'balanced_accuracy': balanced_accuracy_score(targets, predictions)
                }
            except Exception as e2:
                print(f"Error calculating basic metrics: {str(e2)}")
                metrics = {'error': str(e)}

        return metrics

    def _objective(self, trial: optuna.Trial) -> float:

        params = {

            "num_layers": trial.suggest_int("num_layers", 1, 3),

            "base_channels": trial.suggest_int("base_channels", 16, 256, step=16),

            "dropout": trial.suggest_float("dropout", 0.2, 0.6),

            "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),

            "weight_decay": trial.suggest_float("weight_decay", 1e-4, 5e-2, log=True),

            "residual": trial.suggest_categorical("residual", [False, True]),

            "activation": trial.suggest_categorical("activation", ["relu", "leaky_relu"]),

            "edge_dropout": trial.suggest_float("edge_dropout", 0.0, 0.3),

            "focal_gamma": trial.suggest_float("focal_gamma", 2.0, 3.0),

            "class_weight_factor": trial.suggest_float("class_weight_factor", 1.0, 20.0),

            "max_neighbors": trial.suggest_int("max_neighbors", 5, 35),
        }
        hidden_channels = []
        aggregation_methods = []
        num_neighbors = []

        for i in range(params["num_layers"]):
            channels = trial.suggest_int(
                f"layer_{i}_channels",
                int(params["base_channels"] * 0.5),
                params["base_channels"]
            )
            hidden_channels.append(channels)
            aggregation_methods.append(
                trial.suggest_categorical(
                    f"aggregation_layer_{i}", ["mean", "max", "sum"]
                )
            )

            num_neighbors.append(trial.suggest_int(
                f"neighbors_layer_{i}",
                5,
                params["max_neighbors"]
            ))

        params.update({
            "hidden_channels": hidden_channels,
            "aggregation_methods": aggregation_methods,
            "num_neighbors": num_neighbors
        })

        train_timesteps = sorted([ts for ts in self.df_features['Time step'].unique() if 1 <= ts <= 36])
        val_timesteps = sorted([ts for ts in self.df_features['Time step'].unique() if 37 <= ts <= 41])

        print(f"Training on timesteps {train_timesteps}")
        print(f"Validating on timesteps {val_timesteps}")
        model = AdvancedGraphSAGE(
            in_channels=self.input_dim,
            hidden_channels=params["hidden_channels"],
            out_channels=2,
            dropout=params["dropout"],
            residual=params["residual"],
            aggregation_methods=params["aggregation_methods"],
            edge_dropout=params["edge_dropout"]
        ).to(self.device)

        param_device = next(model.parameters()).device
        print(f"Model parameter device: {param_device}")
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=params["learning_rate"],
            weight_decay=params["weight_decay"]
        )
        class_weights = self.class_weights.clone()
        class_weights[1] = class_weights[1] * params["class_weight_factor"]

        best_val_score = -float('inf')
        patience = 8
        patience_counter = 0

        try:

            for epoch in range(50):

                model.train()
                total_train_loss = 0.0
                timesteps_processed = 0

                for timestep in train_timesteps:
                    torch.cuda.empty_cache()

                    timestep_data = self.data_handler.process_timestep_with_sampling(
                        self.df_edges,
                        self.df_features,
                        timestep,
                        params["num_neighbors"]
                    ).to(self.device)

                    valid_mask = (timestep_data.y != -1)
                    if valid_mask.sum() == 0:
                        del timestep_data
                        continue
                    optimizer.zero_grad()
                    out = model(timestep_data.x, timestep_data.edge_index)

                    loss = self.calculate_enhanced_loss(
                        out[valid_mask],
                        timestep_data.y[valid_mask],
                        gamma=params["focal_gamma"],
                        label_smoothing=0.1,
                        class_weights=class_weights
                    )

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    total_train_loss += loss.item()
                    timesteps_processed += 1

                    del timestep_data, out, loss

                avg_train_loss = total_train_loss / max(1, timesteps_processed)

                model.eval()
                val_metrics_by_timestep = []

                with torch.no_grad():
                    for val_timestep in val_timesteps:
                        torch.cuda.empty_cache()
                        val_data = self.data_handler.process_timestep_with_sampling(
                            self.df_edges,
                            self.df_features,
                            val_timestep,
                            params["num_neighbors"]
                        ).to(self.device)

                        val_mask = (val_data.y != -1)
                        if val_mask.sum() == 0:
                            continue

                        val_out = model(val_data.x, val_data.edge_index)
                        val_probs = F.softmax(val_out[val_mask], dim=1)
                        val_preds = val_probs.argmax(dim=1)

                        y_val = val_data.y[val_mask].cpu().numpy()
                        val_preds_np = val_preds.cpu().numpy()
                        val_probs_np = val_probs.cpu().numpy()

                        if len(np.unique(y_val)) > 1:
                            timestep_metrics = self.calculate_metrics(y_val, val_preds_np, val_probs_np)
                            val_metrics_by_timestep.append(timestep_metrics)

                        del val_data, val_out, val_probs, val_preds

                if val_metrics_by_timestep:
                    avg_val_score = np.mean([
                        metrics.get('composite_score', 0) for metrics in val_metrics_by_timestep
                    ])

                    if avg_val_score > best_val_score:
                        best_val_score = avg_val_score
                        patience_counter = 0
                    else:
                        patience_counter += 1

                    if patience_counter >= patience:
                        print(f"Early stopping at epoch {epoch}")
                        break

                    if epoch % 10 == 0:
                        print(f"Epoch {epoch}, train loss: {avg_train_loss:.4f}, val score: {avg_val_score:.4f}")

            print(f"Best validation score: {best_val_score:.4f}")
            return best_val_score

        except Exception as e:
            print(f"Error during trial: {str(e)}")
            return -float('inf')

        del model, optimizer
        torch.cuda.empty_cache()
        gc.collect()

    def train_final_model_with_timesteps(self, best_params: Dict) -> Tuple[nn.Module, Dict]:
        print("\nTraining final model with best parameters...")

        num_layers = best_params["num_layers"]
        dropout = best_params["dropout"]
        learning_rate = best_params["learning_rate"]
        weight_decay = best_params["weight_decay"]*2.0
        residual = best_params["residual"]
        edge_dropout = best_params.get("edge_dropout", 0.0)*1.5
        focal_gamma = best_params.get("focal_gamma", 2.0)
        class_weight_factor = best_params.get("class_weight_factor", 1.0)

        hidden_channels = [best_params[f"layer_{i}_channels"] for i in range(num_layers)]
        aggregation_methods = [best_params[f"aggregation_layer_{i}"] for i in range(num_layers)]
        num_neighbors = [best_params[f"neighbors_layer_{i}"] for i in range(num_layers)]

        all_timesteps = sorted(self.df_features['Time step'].unique())
        train_timesteps = [ts for ts in all_timesteps if 1 <= ts <= 36]
        val_timesteps = [ts for ts in all_timesteps if 37 <= ts <= 41]
        test_timesteps = [ts for ts in all_timesteps if 42 <= ts <= 49]

        print(f"Using timesteps {train_timesteps} for training")
        print(f"Using timesteps {val_timesteps} for validation/threshold optimization")
        print(f"Using timesteps {test_timesteps} for final evaluation")

        model = AdvancedGraphSAGE(
            in_channels=self.input_dim,
            hidden_channels=hidden_channels,
            out_channels=2,
            dropout=dropout,
            residual=residual,
            aggregation_methods=aggregation_methods,
            edge_dropout=edge_dropout
        ).to(self.device)

        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay
        )

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=10, min_lr=1e-6, verbose=True
        )

        scaled_class_weights = self.class_weights.clone()
        scaled_class_weights[1] = scaled_class_weights[1] * class_weight_factor

        print(f"\nTraining with hyperparameters:")
        print(f"  Learning rate: {learning_rate}")
        print(f"  Weight decay: {weight_decay}")
        print(f"  Edge dropout: {edge_dropout}")
        print(f"  Focal gamma: {focal_gamma}")
        print(f"  Class weight factor: {class_weight_factor}")
        print(f"  Original class weights: {self.class_weights}")
        print(f"  Scaled class weights: {scaled_class_weights}")

        model.optimal_class_weights = scaled_class_weights
        model.optimal_gamma = focal_gamma


        best_val_score = -float('inf')
        best_model_state = None
        patience = 15
        patience_counter = 0

        for epoch in range(200):

            model.train()
            total_train_loss = 0.0
            timesteps_processed = 0

            for timestep in train_timesteps:
                torch.cuda.empty_cache()

                timestep_data = self.data_handler.process_timestep_with_sampling(
                    self.df_edges,
                    self.df_features,
                    timestep,
                    num_neighbors
                ).to(self.device)

                valid_mask = (timestep_data.y != -1)
                if valid_mask.sum() == 0:
                    continue

                optimizer.zero_grad()

                # Use edge weights if available
                if hasattr(timestep_data, 'edge_weight'):
                    out = model(timestep_data.x, timestep_data.edge_index, timestep_data.edge_weight)
                else:
                    out = model(timestep_data.x, timestep_data.edge_index)

                # Use focal loss with tuned gamma parameter
                loss = self.calculate_enhanced_loss(
                    out[valid_mask],
                    timestep_data.y[valid_mask],
                    gamma=focal_gamma,
                    label_smoothing=0.1,
                    class_weights=scaled_class_weights
                )

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_train_loss += loss.item()
                timesteps_processed += 1
                del timestep_data, out, loss

            avg_train_loss = total_train_loss / max(1, timesteps_processed)

            model.eval()
            val_metrics_by_timestep = []

            with torch.no_grad():
                for val_timestep in val_timesteps:
                    torch.cuda.empty_cache()
                    val_data = self.data_handler.process_timestep_with_sampling(
                        self.df_edges,
                        self.df_features,
                        val_timestep,
                        num_neighbors
                    ).to(self.device)

                    val_mask = (val_data.y != -1)
                    if val_mask.sum() == 0:
                        continue

                    if hasattr(val_data, 'edge_weight'):
                        val_out = model(val_data.x, val_data.edge_index, val_data.edge_weight)
                    else:
                        val_out = model(val_data.x, val_data.edge_index)

                    val_out = val_out[val_mask]
                    val_probs = F.softmax(val_out, dim=1)
                    val_preds = val_probs.argmax(dim=1)

                    y_val = val_data.y[val_mask].cpu().numpy()
                    val_preds_np = val_preds.cpu().numpy()
                    val_probs_np = val_probs.cpu().numpy()

                    if len(np.unique(y_val)) > 1:
                        timestep_metrics = self.calculate_metrics(y_val, val_preds_np, val_probs_np)
                        val_metrics_by_timestep.append(timestep_metrics)
                    del val_data, val_out, val_probs, val_preds

            if val_metrics_by_timestep:
                avg_composite_score = np.mean([
                    metrics.get('composite_score', 0) for metrics in val_metrics_by_timestep
                ])

                if avg_composite_score > best_val_score:
                    best_val_score = avg_composite_score
                    best_model_state = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                else:
                    patience_counter += 1

                scheduler.step(avg_composite_score)

                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break

                if epoch % 5 == 0:
                    avg_metrics = {
                        k: np.mean([m.get(k, 0) for m in val_metrics_by_timestep])
                        for k in val_metrics_by_timestep[0].keys()
                        if k != 'confusion_matrix'
                    }
                    print(f"Epoch {epoch}: Avg Train Loss = {avg_train_loss:.4f}, Val Score = {avg_composite_score:.4f}")
                    for k, v in avg_metrics.items():
                        if k in ['composite_score', 'class_1_recall', 'class_1_precision', 'class_1_f1']:
                            print(f"  {k}: {v:.4f}")

                    if self.wandb_run is not None:
                        log_dict = {
                            'epoch': epoch,
                            'train_loss': avg_train_loss,
                            'val_composite_score': avg_composite_score,
                            **{f'val_{k}': v for k, v in avg_metrics.items()}
                        }
                        self.wandb_run.log(log_dict)

        print("\nLoading best model state from training")
        model.load_state_dict(best_model_state)


        print("\n=== Finding Optimal Classification Threshold ===")
        val_probs_all = []
        val_true_all = []

        model.eval()
        with torch.no_grad():
            for val_timestep in val_timesteps:
                torch.cuda.empty_cache()
                val_data = self.data_handler.process_timestep_with_sampling(
                    self.df_edges,
                    self.df_features,
                    val_timestep,
                    num_neighbors
                ).to(self.device)

                val_mask = (val_data.y != -1)
                if val_mask.sum() == 0:
                    continue

                if hasattr(val_data, 'edge_weight'):
                    val_out = model(val_data.x, val_data.edge_index, val_data.edge_weight)
                else:
                    val_out = model(val_data.x, val_data.edge_index)

                val_probs = F.softmax(val_out[val_mask], dim=1)
                val_true_all.append(val_data.y[val_mask].cpu().numpy())
                val_probs_all.append(val_probs.cpu().numpy())
                del val_data, val_out, val_probs

        if val_probs_all and len(val_probs_all) > 0:
            val_probs_combined = np.vstack(val_probs_all)
            val_true_combined = np.concatenate(val_true_all)
            optimal_threshold = self.find_optimal_threshold(val_true_combined, val_probs_combined[:, 1])
            print(f"\nOptimal threshold found: {optimal_threshold:.4f}")
        else:
            optimal_threshold = 0.5
            print("\nUsing default threshold: 0.5 (no valid validation data)")

        model.optimal_threshold = optimal_threshold

        print("\n=== Evaluating Model Performance Per Timestep Starting from 37 (Using Adjusted Threshold) ===")

        val_timesteps = [ts for ts in self.df_features['Time step'].unique() if 37 <= ts <= 41]
        test_timesteps = [ts for ts in self.df_features['Time step'].unique() if 42 <= ts <= 49]
        all_eval_timesteps = sorted(val_timesteps + test_timesteps)

        test_timestep_results = {}
        for eval_timestep in all_eval_timesteps:
            print(f"\nEvaluating on timestep {eval_timestep}:")

            timestep_type = "validation" if eval_timestep <= 41 else "test"
            print(f"  Timestep type: {timestep_type}")
r
            timestep_data = self.data_handler.process_timestep_with_sampling(
                self.df_edges,
                self.df_features,
                eval_timestep,
                num_neighbors
            ).to(self.device)

            valid_mask = (timestep_data.y != -1)
            if valid_mask.sum() == 0:
                print(f"  No valid nodes for timestep {eval_timestep}")
                continue

            model.eval()
            with torch.no_grad():
                out = model(timestep_data.x, timestep_data.edge_index)
                out_valid = out[valid_mask]
                probs = F.softmax(out_valid, dim=1)

                y_true = timestep_data.y[valid_mask].cpu().numpy()
                probs_np = probs.cpu().numpy()

                preds_np = (probs_np[:, 1] >= optimal_threshold).astype(int)

                try:

                    test_metrics = self.calculate_metrics(y_true, preds_np, probs_np)
                    test_timestep_results[eval_timestep] = test_metrics


                    print(f"  Composite Score: {test_metrics.get('composite_score', 0):.4f}")
                    print(f"  Class 1 Recall: {test_metrics.get('class_1_recall', 0):.4f}")
                    print(f"  Class 1 F1: {test_metrics.get('class_1_f1', 0):.4f}")
                    print(f"  Class 0 Recall: {test_metrics.get('class_0_recall', 0):.4f}")
                    print(f"  Class 0 F1: {test_metrics.get('class_0_f1', 0):.4f}")
                    print(f"  Balanced Acc: {test_metrics.get('balanced_accuracy', 0):.4f}")
                    print(f"  Precision: {test_metrics.get('precision', 0):.4f}")
                    print(f"  MCC: {test_metrics.get('mcc', 0):.4f}")
                    print(f"  PR-AUC: {test_metrics.get('pr_auc', 0):.4f}")
                    print(f"  ROC-AUC: {test_metrics.get('auroc', 0):.4f}")

                    if self.wandb_run is not None:
                        self.wandb_run.log({
                            f"timestep_{eval_timestep}_composite_score": test_metrics.get('composite_score', 0),
                            f"timestep_{eval_timestep}_recall_1": test_metrics.get('class_1_recall', 0),
                            f"timestep_{eval_timestep}_f1_1": test_metrics.get('class_1_f1', 0),
                            f"timestep_{eval_timestep}_recall_0": test_metrics.get('class_0_recall', 0),
                            f"timestep_{eval_timestep}_f1_0": test_metrics.get('class_0_f1', 0),
                            f"timestep_{eval_timestep}_balanced_acc": test_metrics.get('balanced_accuracy', 0),
                            f"timestep_{eval_timestep}_precision": test_metrics.get('precision', 0),
                            f"timestep_{eval_timestep}_mcc": test_metrics.get('mcc', 0),
                            f"timestep_{eval_timestep}_pr_auc": test_metrics.get('pr_auc', 0),
                            f"timestep_{eval_timestep}_roc_auc": test_metrics.get('auroc', 0)
                        })

                except Exception as e:
                    print(f"  Error calculating metrics: {str(e)}")
                    print(f"  Debug - y_true shape: {y_true.shape}, preds_np shape: {preds_np.shape}")
                    print(f"  Debug - probs_np shape: {probs_np.shape}")
                    print(f"  Debug - unique values in y_true: {np.unique(y_true)}")
                    print(f"  Debug - unique values in preds_np: {np.unique(preds_np)}")

        print("\n=== Final Evaluation on Test Set (Default Threshold) ===")
        test_metrics_default = self.evaluate_model_on_data(
            model=model,
            timesteps=test_timesteps,
            num_neighbors=num_neighbors,
            threshold=0.5
        )

        if test_metrics_default:
            print("Final Test Metrics (Default Threshold):")
            for metric, value in test_metrics_default.items():
                if metric != 'combined_metrics' and metric != 'confusion_matrix':
                    print(f"{metric}: {value:.4f}")
                elif metric == 'confusion_matrix':
                    print(f"{metric}: {value}")

        print(f"\nFinal Test Metrics (Adjusted Threshold = {optimal_threshold:.4f}):")
        test_metrics_adjusted = self.evaluate_model_on_data(
            model=model,
            timesteps=test_timesteps,
            num_neighbors=num_neighbors,
            threshold=optimal_threshold
        )

        if test_metrics_adjusted:
            for metric, value in test_metrics_adjusted.items():
                if metric != 'combined_metrics' and metric != 'confusion_matrix':
                    print(f"{metric}: {value:.4f}")
                elif metric == 'confusion_matrix':
                    print(f"{metric}: {value}")

        print("\nThreshold Adjustment Impact:")
        if test_metrics_default and test_metrics_adjusted:
            for metric in test_metrics_default.keys():
                if metric != 'combined_metrics' and metric != 'confusion_matrix':
                    default_val = test_metrics_default.get(metric, 0)
                    adjusted_val = test_metrics_adjusted.get(metric, 0)
                    diff = adjusted_val - default_val
                    print(f"{metric}: {diff:.4f} ({'improvement' if diff > 0 else 'decrease'})")

        if self.wandb_run is not None:
            self.wandb_run.log({
                **{f'test_default_{k}': v for k, v in test_metrics_default.items()
                  if k != 'combined_metrics' and k != 'confusion_matrix'},
                **{f'test_adjusted_{k}': v for k, v in test_metrics_adjusted.items()
                  if k != 'combined_metrics' and k != 'confusion_matrix'},
                'optimal_threshold': optimal_threshold
            })

        return model, {
            'val_score': best_val_score,
            'optimal_threshold': optimal_threshold,
            'timestep_results': test_timestep_results,
            'test_metrics_default': test_metrics_default,
            'test_metrics_adjusted': test_metrics_adjusted
        }

    def evaluate_model_on_data(self, model, timesteps, num_neighbors, threshold=0.5):
        model.eval()
        metrics_by_timestep = []
        all_true = []
        all_pred = []
        all_probs = []

        with torch.no_grad():
            for timestep in timesteps:
                torch.cuda.empty_cache()
                data = self.data_handler.process_timestep_with_sampling(
                    self.df_edges,
                    self.df_features,
                    timestep,
                    num_neighbors
                ).to(self.device)

                valid_mask = (data.y != -1)
                if valid_mask.sum() == 0:
                    continue

                if hasattr(data, 'edge_weight'):
                    out = model(data.x, data.edge_index, data.edge_weight)
                else:
                    out = model(data.x, data.edge_index)

                probs = F.softmax(out[valid_mask], dim=1)
                preds = (probs[:, 1] >= threshold).long()

                y_true = data.y[valid_mask].cpu().numpy()
                preds_np = preds.cpu().numpy()
                probs_np = probs.cpu().numpy()

                all_true.append(y_true)
                all_pred.append(preds_np)
                all_probs.append(probs_np)

                if len(np.unique(y_true)) > 1:
                    metrics = self.calculate_metrics(y_true, preds_np, probs_np)
                    metrics['timestep'] = timestep
                    metrics_by_timestep.append(metrics)

        if metrics_by_timestep:
            avg_metrics = {
                k: np.mean([m.get(k, 0) for m in metrics_by_timestep])
                for k in metrics_by_timestep[0].keys()
                if k != 'timestep' and k != 'confusion_matrix'
            }

            if all_true and all_pred and all_probs:
                combined_true = np.concatenate(all_true)
                combined_pred = np.concatenate(all_pred)
                combined_probs = np.vstack(all_probs)

                combined_metrics = self.calculate_metrics(combined_true, combined_pred, combined_probs)
                avg_metrics['combined_metrics'] = combined_metrics

            return avg_metrics
        else:
            return {}

    def calculate_enhanced_loss(self, outputs, targets, gamma=2.0,
                              label_smoothing=0.1, class_weights=None):
        # Apply label smoothing
        num_classes = outputs.size(-1)
        smooth_targets = torch.zeros_like(outputs)
        smooth_targets.fill_(label_smoothing / (num_classes - 1))
        smooth_targets.scatter_(1, targets.unsqueeze(1), 1 - label_smoothing)

        # Calculate focal loss
        probs = F.softmax(outputs, dim=-1)
        focal_weight = (1 - probs) ** gamma

        loss = -(focal_weight * smooth_targets * torch.log(probs + 1e-8))
        loss = loss.sum(dim=1)

        if class_weights is not None:
            class_weights = class_weights.to(outputs.device)
            weight_per_sample = class_weights[targets]
            loss = loss * weight_per_sample

        return loss.mean()

    def run_parameter_search(self, n_trials: int = 50) -> Dict:

        study = optuna.create_study(
            direction="maximize",
            pruner=optuna.pruners.HyperbandPruner(
                min_resource=5,
                max_resource=50,
                reduction_factor=3
            )
        )
        study.optimize(self._objective, n_trials=n_trials)

        print("\nBest parameters found:")
        for param, value in study.best_params.items():
            print(f"{param}: {value}")

        if self.wandb_run is not None:
            self.wandb_run.log({
                "best_params": study.best_params,
                "hyperparameter_search": "fixed_validation_set"
            })

        return study.best_params


    def cleanup(self):
        """Cleanup resources"""
        if self.wandb_run is not None:
            try:
                self.wandb_run.finish()
            except:
                pass
            self.wandb_run = None



In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive'
save_path = '/content/drive/My Drive'
files = os.listdir(data_path)
print(files)

['Colab Notebooks', 'AddrAddr_edgelist.csv', 'wallets_features_classes_combined.csv', 'processed_dataset_with_network.csv', 'processed_data']


In [ ]:
class DataHandler:

    def __init__(self, checkpoint_dir: str, device=None):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.node_maps = {}
        self.edge_indices = {}
        self.data_cache = {}
        self.oversampled_data_cache = {}
        self.cache_hits = 0
        self.cache_misses = 0

        self.device = device if device is not None else torch.device('cpu')

    def process_timestep_with_oversampling(self, df_edges: pd.DataFrame, df_features: pd.DataFrame,
                                         timestep: int, num_neighbors: List[int],
                                         oversample_ratio: float = 30.0) -> Data:

        cache_key = f"oversample_ts{timestep}_ratio{oversample_ratio}_nn{hash(str(num_neighbors))}"
        if cache_key in self.oversampled_data_cache:

            self.cache_hits += 1
            cached_data = self.oversampled_data_cache[cache_key]
            if self.cache_hits % 50 == 0:
                print(f"Cache hit: {cache_key} (hits: {self.cache_hits}, misses: {self.cache_misses})")
            return cached_data.to(self.device)

        self.cache_misses += 1
        if self.cache_misses % 10 == 0:
            print(f"Cache miss: {cache_key} (hits: {self.cache_hits}, misses: {self.cache_misses})")

        data = self.process_timestep_with_sampling(df_edges, df_features, timestep, num_neighbors)
        if timestep < 1 or timestep > 36:
            return data

        valid_mask = (data.y != -1)
        if valid_mask.sum() > 0:
            try:

                valid_labels = data.y[valid_mask].cpu()
                class_counts = torch.bincount(valid_labels)

                if len(class_counts) > 1 and class_counts[1] > 0:
                    majority_count = class_counts[0].item()
                    minority_count = class_counts[1].item()
                    target_minority = int(min(minority_count * oversample_ratio, majority_count))
                    n_synthetic = max(0, target_minority - minority_count)

                    if n_synthetic > 0:
                        print(f"Timestep {timestep}: Oversampling minority class. Original: {minority_count}, Target: {target_minority}")

                        new_x, new_edge_index, new_y = graph_smote(
                            data.x, data.edge_index, data.y,
                            minority_class=1,
                            k_neighbors=5,
                            n_synthetic=n_synthetic
                        )

                        if new_x.size(0) > data.x.size(0):
                            new_nodes_count = new_x.size(0) - data.x.size(0)
                            new_time_steps = torch.full(
                                (new_nodes_count,), timestep,
                                dtype=torch.long, device=data.time_step.device
                            )
                            new_time_step = torch.cat([data.time_step, new_time_steps])

                            if hasattr(data, 'original_indices'):
                                synthetic_indices = torch.full(
                                    (new_nodes_count,), -1,
                                    dtype=data.original_indices.dtype,
                                    device=data.original_indices.device
                                )
                                new_original_indices = torch.cat([data.original_indices, synthetic_indices])
                            else:
                                new_original_indices = None

                            data = Data(
                                x=new_x,
                                edge_index=new_edge_index,
                                y=new_y,
                                time_step=new_time_step,
                                original_indices=new_original_indices if hasattr(data, 'original_indices') else None
                            )

                            data.edge_weight = weight_minority_edges(
                                data.edge_index, data.y,
                                minority_class=1, weight_factor=2.0
                            )

                            print(f"  Created {new_nodes_count} synthetic nodes.")
            except Exception as e:
                print(f"Error in oversampling timestep {timestep}: {str(e)}")

        self.oversampled_data_cache[cache_key] = data.cpu()
        return data.to(self.device)

    def get_cache_stats(self):
        total_calls = self.cache_hits + self.cache_misses
        hit_rate = self.cache_hits / total_calls if total_calls > 0 else 0

        return {
            "hits": self.cache_hits,
            "misses": self.cache_misses,
            "total_calls": total_calls,
            "hit_rate": hit_rate,
            "oversampled_entries": len(self.oversampled_data_cache),
            "standard_entries": len(self.data_cache)
        }

    @staticmethod
    def preprocess_with_smote(df_features, df_edges, oversample_ratio=30.0, start_timestep=1, end_timestep=35):

        print(f"\n=== Preprocessing with GraphSMOTE (Timesteps {start_timestep}-{end_timestep}) ===")
        features_df = df_features.copy()
        edges_df = df_edges.copy()
        max_node_id = features_df['node_id'].max()
        if isinstance(max_node_id, str):
            max_node_id = max([int(nid.split('_')[-1]) if '_' in nid else 0
                              for nid in features_df['node_id']])
        next_node_id = max_node_id + 1
        new_nodes = []
        new_edges = []
        all_timesteps = sorted(features_df['Time step'].unique())
        allowed_timesteps = [ts for ts in all_timesteps if start_timestep <= ts <= end_timestep]

        print(f"Will only process timesteps: {allowed_timesteps}")

        for timestep in allowed_timesteps:
            timestep_features = features_df[features_df['Time step'] == timestep]
            if 'binary_class' in timestep_features.columns:
                minority_nodes = timestep_features[timestep_features['binary_class'] == 1]
                majority_nodes = timestep_features[timestep_features['binary_class'] == 0]
            else:
                minority_nodes = timestep_features[timestep_features['class'] == 1]
                majority_nodes = timestep_features[timestep_features['class'] == 2]d
            if len(minority_nodes) == 0 or len(minority_nodes) >= len(majority_nodes):
                print(f"Timestep {timestep}: Skipping (minority: {len(minority_nodes)}, majority: {len(majority_nodes)})")
                continue

            target_samples = min(int(len(minority_nodes) * oversample_ratio), len(majority_nodes))
            samples_to_generate = max(0, target_samples - len(minority_nodes))

            print(f"Timestep {timestep}: Oversampling minority class. Original: {len(minority_nodes)}, Target: {target_samples}")

            if samples_to_generate == 0:
                continue

            node_to_idx = {node: idx for idx, node in enumerate(timestep_features['node_id'])}
            idx_to_node = {idx: node for node, idx in node_to_idx.items()}

            feature_cols = timestep_features.columns.difference(['node_id', 'Time step', 'class', 'binary_class'])
            X = torch.tensor(timestep_features[feature_cols].values, dtype=torch.float32)
            if 'binary_class' in timestep_features.columns:
                y = torch.tensor(timestep_features['binary_class'].values, dtype=torch.long)
            else:
                y = torch.tensor(timestep_features['class'].map(lambda x: 1 if x == 1 else (0 if x == 2 else -1)).values,
                                dtype=torch.long)

            timestep_edges = edges_df[
                edges_df['source'].isin(timestep_features['node_id']) &
                edges_df['target'].isin(timestep_features['node_id'])
            ]

            edge_index = []
            for _, edge in timestep_edges.iterrows():
                if edge['source'] in node_to_idx and edge['target'] in node_to_idx:
                    edge_index.append([node_to_idx[edge['source']], node_to_idx[edge['target']]])

            edge_index = torch.tensor(edge_index, dtype=torch.long).t() if edge_index else torch.zeros((2, 0), dtype=torch.long)

            try:
                new_X, new_edge_index, new_y = graph_smote(
                    X, edge_index, y,
                    minority_class=1,
                    k_neighbors=5,
                    n_synthetic=samples_to_generate
                )

                num_original_nodes = X.size(0)
                num_synthetic_nodes = new_X.size(0) - num_original_nodes

                if num_synthetic_nodes > 0:
                    print(f"  Created {num_synthetic_nodes} synthetic nodes.")

                    for i in range(num_original_nodes, new_X.size(0)):
                        if isinstance(max_node_id, str):
                            new_id = f"synthetic_{next_node_id}"
                        else:
                            new_id = f"synthetic_{next_node_id}"
                        next_node_id += 1
                        node_data = {
                            'node_id': new_id,
                            'Time step': timestep,
                            'binary_class': new_y[i].item(),
                            'class': 1 if new_y[i].item() == 1 else 2,
                            **{col: new_X[i, j].item() for j, col in enumerate(feature_cols)}
                        }
                        new_nodes.append(node_data)
                        idx_to_node[i] = new_id

                    for j in range(new_edge_index.size(1)):
                        src_idx, dst_idx = new_edge_index[0, j].item(), new_edge_index[1, j].item()

                        if src_idx >= num_original_nodes or dst_idx >= num_original_nodes:
                            if src_idx in idx_to_node and dst_idx in idx_to_node:
                                new_edges.append({
                                    'source': idx_to_node[src_idx],
                                    'target': idx_to_node[dst_idx]
                                })

            except Exception as e:
                print(f"Error in GraphSMOTE for timestep {timestep}: {e}")
                continue

        if new_nodes:
            synthetic_nodes_df = pd.DataFrame(new_nodes)
            oversampled_features = pd.concat([features_df, synthetic_nodes_df], ignore_index=True)
            print(f"Added {len(synthetic_nodes_df)} synthetic nodes to features DataFrame.")
        else:
            oversampled_features = features_df
            print("No synthetic nodes were added.")


        if new_edges:
            synthetic_edges_df = pd.DataFrame(new_edges)
            oversampled_edges = pd.concat([edges_df, synthetic_edges_df], ignore_index=True)
            print(f"Added {len(synthetic_edges_df)} synthetic edges to edges DataFrame.")
        else:
            oversampled_edges = edges_df
            print("No synthetic edges were added.")

        if 'binary_class' in oversampled_features.columns:
            print("\nClass distribution after oversampling:")
            print(oversampled_features['binary_class'].value_counts().sort_index())
        else:
            print("\nClass distribution after oversampling:")
            print(oversampled_features['class'].value_counts().sort_index())

        print("\n=== Summary of GraphSMOTE Preprocessing ===")
        print(f"Only processed timesteps {start_timestep} to {end_timestep}")
        if new_nodes:
            print(f"Total synthetic nodes added: {len(new_nodes)}")
        if new_edges:
            print(f"Total synthetic edges added: {len(new_edges)}")

        return oversampled_features, oversampled_edges

    def load_preprocessed_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        df_features = pd.read_parquet(self.checkpoint_dir / 'processed_features.parquet')
        df_edges = pd.read_parquet(self.checkpoint_dir / 'processed_edges.parquet')

        df_features['node_id'] = df_features['node_id'].astype(str)
        df_edges['source'] = df_edges['source'].astype(str)
        df_edges['target'] = df_edges['target'].astype(str)

        self.valid_nodes = set(df_features['node_id'])

        return df_edges, df_features

    def process_timestep_with_sampling(self, df_edges: pd.DataFrame, df_features: pd.DataFrame,
                                    timestep: int, num_neighbors: List[int]) -> Data:

        cache_key = f"timestep_{timestep}_{hash(str(num_neighbors))}"
        if cache_key in self.data_cache:
            cached_data = self.data_cache[cache_key]
            return cached_data.to(self.device)
        current_features = df_features[df_features['Time step'] == timestep]
        if timestep not in self.node_maps:
            self.node_maps[timestep] = {node: idx for idx, node in enumerate(current_features['node_id'])}
        node_map = self.node_maps[timestep]

        feature_cols = current_features.columns.difference(['node_id', 'Time step', 'class', 'binary_class'])
        x = torch.tensor(current_features[feature_cols].values, dtype=torch.float32)

        if timestep not in self.edge_indices:
            self.edge_indices[timestep] = self._create_edge_index(df_edges, node_map)
        edge_index = self.edge_indices[timestep]

        if 'binary_class' in current_features.columns:
            y = torch.tensor(current_features['binary_class'].values, dtype=torch.long)
        else:
            y = torch.tensor(current_features['class'].values, dtype=torch.long)

        data = Data(
            x=x.to(self.device),
            edge_index=edge_index.to(self.device),
            y=y.to(self.device),
            time_step=torch.full((len(x),), timestep, dtype=torch.long).to(self.device),
            original_indices=torch.arange(len(current_features)).to(self.device)
        )

        subgraph_loader = NeighborLoader(
            data,
            num_neighbors=num_neighbors,
            batch_size=len(data.x),
            shuffle=True,
            directed=True
        )

        batch = next(iter(subgraph_loader))

        return batch

    def _create_edge_index(self, df_edges: pd.DataFrame, node_map: Dict) -> torch.Tensor:
        valid_nodes_subset = set(node_map.keys())

        valid_source = df_edges['source'].isin(valid_nodes_subset)
        valid_target = df_edges['target'].isin(valid_nodes_subset)
        mask = valid_source & valid_target
        valid_edges = df_edges[mask]

        src_indices = [node_map[src] for src in valid_edges['source']]
        dst_indices = [node_map[dst] for dst in valid_edges['target']]

        edge_index = torch.tensor([src_indices, dst_indices], dtype=torch.long)

        return edge_index



In [ ]:

class MetricsVisualizer:
    def __init__(self, wandb_run=None):
        self.wandb_run = wandb_run

    def plot_confusion_matrix(self, y_true: np.ndarray, y_pred: np.ndarray, title: str) -> plt.Figure:
        cm = confusion_matrix(y_true, y_pred)
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        ax.set_title(title)
        return fig

    def plot_pr_curve(self, y_true: np.ndarray, y_prob: np.ndarray, title: str) -> plt.Figure:
        precision, recall, _ = precision_recall_curve(y_true, y_prob)
        pr_auc = auc(recall, precision)

        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(recall, precision, label=f'PR-AUC: {pr_auc:.3f}')
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.set_title(title)
        ax.legend()
        return fig

    def log_visualizations(self, y_true: np.ndarray, y_pred: np.ndarray,
                          y_prob: np.ndarray, prefix: str = ''):
        if self.wandb_run is not None:
            cm_fig = self.plot_confusion_matrix(y_true, y_pred, f'{prefix} Confusion Matrix')
            pr_fig = self.plot_pr_curve(y_true, y_prob, f'{prefix} Precision-Recall Curve')

            self.wandb_run.log({
                f'{prefix}_confusion_matrix': wandb.Image(cm_fig),
                f'{prefix}_pr_curve': wandb.Image(pr_fig)
            })

            plt.close(cm_fig)
            plt.close(pr_fig)

In [ ]:
class AdvancedGraphSAGE(nn.Module):
    def __init__(self, in_channels: int, hidden_channels: List[int], out_channels: int,
                 dropout: float = 0.5, residual: bool = True,
                 aggregation_methods: List[str] = None, edge_dropout: float = 0.0):
        super().__init__()
        self.num_layers = len(hidden_channels)
        self.residual = residual
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.dropout_rate = dropout
        self.edge_dropout_rate = edge_dropout

        if aggregation_methods is None:
            aggregation_methods = ['mean'] * self.num_layers

        self.layers = nn.ModuleList([
            CustomSAGEConv(in_channels, hidden_channels[0], aggr=aggregation_methods[0])
        ])

        for i in range(1, self.num_layers):
            self.layers.append(
                CustomSAGEConv(
                    hidden_channels[i-1],
                    hidden_channels[i],
                    aggr=aggregation_methods[i]
                )
            )

        if self.residual:
            self.projections = nn.ModuleList()
            for i in range(len(hidden_channels)-1):
                self.projections.append(nn.Sequential(
                    nn.Linear(hidden_channels[i], hidden_channels[i+1]),
                    nn.BatchNorm1d(hidden_channels[i+1])
                ))

        self.linear = nn.Linear(hidden_channels[-1], out_channels)
        self.dropouts = nn.ModuleList([
            nn.Dropout(dropout) for _ in range(self.num_layers)
        ])

        self.batch_norms = nn.ModuleList([
            nn.BatchNorm1d(hidden_dim) for hidden_dim in hidden_channels
        ])

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

    def forward(self, x, edge_index, edge_weight=None):
        if edge_weight is None:
            edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)

        if self.training and self.edge_dropout_rate > 0:
            edge_mask = torch.rand(edge_index.size(1), device=edge_index.device) >= self.edge_dropout_rate
            edge_index = edge_index[:, edge_mask]
            edge_weight = edge_weight[edge_mask]

        residuals = []
        x = self.layers[0](x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.batch_norms[0](x)
        x = self.dropouts[0](x)
        residuals.append(x)
        for i in range(1, self.num_layers):
            h = self.layers[i](x, edge_index, edge_weight)
            h = F.relu(h)
            h = self.batch_norms[i](h)

            if self.residual and i > 0:
                projected = self.projections[i-1](residuals[i-1])
                h = h + projected

            h = self.dropouts[i](h)
            residuals.append(h)
            x = h

        return self.linear(x)

class EdgeDropout(nn.Module):
    def __init__(self, p=0.1):
        super().__init__()
        self.p = p

    def forward(self, edge_index):
        if not self.training or self.p == 0:
            return edge_index
        edge_mask = torch.rand(edge_index.size(1), device=edge_index.device) >= self.p
        return edge_index[:, edge_mask]



In [ ]:
def graph_smote(x, edge_index, y, minority_class=1, k_neighbors=5, n_synthetic=None):

    minority_mask = (y == minority_class)
    majority_mask = (y == 0) & (y != -1)

    minority_indices = torch.nonzero(minority_mask).squeeze()
    majority_indices = torch.nonzero(majority_mask).squeeze()

    if minority_indices.dim() == 0:
        minority_indices = minority_indices.unsqueeze(0)

    if majority_indices.dim() == 0:
        majority_indices = majority_indices.unsqueeze(0)

    if minority_indices.numel() == 0:
        return x, edge_index, y

    if n_synthetic is None:
        n_synthetic = len(majority_indices) - len(minority_indices)

    if n_synthetic <= 0:
        return x, edge_index, y

    row, col = edge_index
    N = x.size(0)

    synthetic_features = []
    new_edges = []

    for _ in range(n_synthetic):
        idx = minority_indices[torch.randint(0, len(minority_indices), (1,))]

        neighbors = []
        last_nodes = {idx.item()}
        visited = set(last_nodes)

        for _ in range(2):
            current = set()
            for node in last_nodes:
                neighbor_indices = col[row == node].tolist()
                same_class_neighbors = [n for n in neighbor_indices
                                       if n not in visited and
                                       n < len(y) and y[n].item() == minority_class]
                current.update(same_class_neighbors)
                neighbors.extend(same_class_neighbors)
            visited.update(current)
            last_nodes = current
            if len(neighbors) >= k_neighbors:
                break

        if len(neighbors) < 2:
            dists = torch.norm(x[idx].unsqueeze(0) - x[minority_indices], dim=1)
            _, nn_indices = torch.topk(dists, min(k_neighbors + 1, len(minority_indices)), largest=False)
            neighbors = [minority_indices[i].item() for i in nn_indices[1:]]

        if not neighbors:
            continue

        neighbor_idx = neighbors[torch.randint(0, len(neighbors), (1,)).item()]
        alpha = torch.rand(1).item()
        synthetic_feature = alpha * x[idx] + (1 - alpha) * x[neighbor_idx]
        synthetic_features.append(synthetic_feature)

        src_neighbors = col[row == idx.item()].tolist()
        dst_neighbors = col[row == neighbor_idx].tolist()
        parent_neighbors = set(src_neighbors + dst_neighbors)

        for neighbor in parent_neighbors:
            new_edges.append((N + len(synthetic_features) - 1, neighbor))
            new_edges.append((neighbor, N + len(synthetic_features) - 1))

    if synthetic_features:
        try:
            print(f"Debug - x shape: {x.shape}, num synthetic features: {len(synthetic_features)}")
            print(f"First synthetic feature shape: {synthetic_features[0].shape}")
            synthetic_features_tensor = torch.stack(synthetic_features)
            if len(x.shape) == 2 and len(synthetic_features_tensor.shape) == 2:
                new_x = torch.cat([x, synthetic_features_tensor], dim=0)
            elif len(x.shape) == 2 and len(synthetic_features_tensor.shape) == 3:
                synthetic_reshaped = synthetic_features_tensor.view(-1, x.shape[1])
                new_x = torch.cat([x, synthetic_reshaped], dim=0)
            else:
                print(f"Warning: Dimension mismatch. x: {x.shape}, synthetic: {synthetic_features_tensor.shape}")
                if x.shape[1:] != synthetic_features_tensor.shape[1:]:
                    reshaped_features = []
                    for feat in synthetic_features:
                        reshaped = feat.view(1, -1)
                        if reshaped.shape[1] != x.shape[1]:
                            if reshaped.shape[1] < x.shape[1]:
                                padding = torch.zeros(1, x.shape[1] - reshaped.shape[1],
                                                     device=reshaped.device)
                                reshaped = torch.cat([reshaped, padding], dim=1)
                            else:
                                reshaped = reshaped[:, :x.shape[1]]
                        reshaped_features.append(reshaped)
                    if reshaped_features:
                        synthetic_features_tensor = torch.cat(reshaped_features, dim=0)
                        new_x = torch.cat([x, synthetic_features_tensor], dim=0)
                    else:
                        print("Warning: Failed to reshape synthetic features, returning original data")
                        return x, edge_index, y

            print(f"Success - new_x shape: {new_x.shape}")
        except Exception as e:
            print(f"Error in graph_smote feature concatenation: {str(e)}")
            print(f"x shape: {x.shape}, synthetic features: {len(synthetic_features)}")
            return x, edge_index, y

        new_labels = torch.full((len(synthetic_features),), minority_class,
                               dtype=y.dtype, device=y.device)
        new_y = torch.cat([y, new_labels])
        if new_edges:
            new_edge_indices = torch.tensor(new_edges, dtype=edge_index.dtype,
                                           device=edge_index.device).t()
            new_edge_index = torch.cat([edge_index, new_edge_indices], dim=1)
        else:
            new_edge_index = edge_index

        return new_x, new_edge_index, new_y

    return x, edge_index, y

def weight_minority_edges(edge_index, y, edge_weights=None, minority_class=1, weight_factor=2.0):

    source, target = edge_index
    if edge_weights is None:
        edge_weights = torch.ones(edge_index.size(1), device=edge_index.device)

    for i in range(edge_index.size(1)):
        src_node, tgt_node = source[i].item(), target[i].item()
        if src_node < len(y) and tgt_node < len(y):

            if y[src_node] == minority_class and y[tgt_node] == minority_class:
                edge_weights[i] = weight_factor * weight_factor

            elif y[src_node] == minority_class or y[tgt_node] == minority_class:
                edge_weights[i] = weight_factor

    return edge_weights

def analyze_oversampling_impact(self, model, test_timesteps, num_neighbors):
    print("\n=== Analyzing Oversampling Impact ===")

    results_with_oversampling = []
    results_without_oversampling = []

    model.eval()
    with torch.no_grad():
        for test_timestep in test_timesteps:
            torch.cuda.empty_cache()

            standard_data = self.data_handler.process_timestep_with_sampling(
                self.df_edges,
                self.df_features,
                test_timestep,
                num_neighbors
            ).to(self.device)

            oversampled_data = self.data_handler.process_timestep_with_sampling(
                self.df_edges,
                self.df_features,
                test_timestep,
                num_neighbors
            ).to(self.device)

            valid_mask_std = (standard_data.y != -1)
            if valid_mask_std.sum() > 0:
                out_std = model(standard_data.x, standard_data.edge_index)
                probs_std = F.softmax(out_std[valid_mask_std], dim=1)
                preds_std = (probs_std[:, 1] >= self.optimal_threshold).long()

                y_true_std = standard_data.y[valid_mask_std].cpu().numpy()
                y_pred_std = preds_std.cpu().numpy()
                y_prob_std = probs_std.cpu().numpy()

                if len(np.unique(y_true_std)) > 1:
                    metrics_std = self.calculate_metrics(y_true_std, y_pred_std, y_prob_std)
                    metrics_std['timestep'] = test_timestep
                    results_without_oversampling.append(metrics_std)

            valid_mask_over = (oversampled_data.y != -1)
            if hasattr(oversampled_data, 'original_indices'):
                valid_mask_over = valid_mask_over & (oversampled_data.original_indices != -1)

            if valid_mask_over.sum() > 0:
                if hasattr(oversampled_data, 'edge_weight'):
                    out_over = model(oversampled_data.x, oversampled_data.edge_index,
                                     oversampled_data.edge_weight)
                else:
                    out_over = model(oversampled_data.x, oversampled_data.edge_index)

                probs_over = F.softmax(out_over[valid_mask_over], dim=1)
                preds_over = (probs_over[:, 1] >= self.optimal_threshold).long()

                y_true_over = oversampled_data.y[valid_mask_over].cpu().numpy()
                y_pred_over = preds_over.cpu().numpy()
                y_prob_over = probs_over.cpu().numpy()

                if len(np.unique(y_true_over)) > 1:
                    metrics_over = self.calculate_metrics(y_true_over, y_pred_over, y_prob_over)
                    metrics_over['timestep'] = test_timestep
                    results_with_oversampling.append(metrics_over)

    if results_without_oversampling and results_with_oversampling:
        print("\nAverage Metrics Without Oversampling:")
        avg_without = {
            key: np.mean([r[key] for r in results_without_oversampling if key in r])
            for key in results_without_oversampling[0].keys()
            if key != 'timestep' and key != 'confusion_matrix'
        }

        for metric, value in avg_without.items():
            print(f"  {metric}: {value:.4f}")

        print("\nAverage Metrics With Oversampling:")
        avg_with = {
            key: np.mean([r[key] for r in results_with_oversampling if key in r])
            for key in results_with_oversampling[0].keys()
            if key != 'timestep' and key != 'confusion_matrix'
        }

        for metric, value in avg_with.items():
            print(f"  {metric}: {value:.4f}")

        print("\nDifference (With - Without):")
        for metric in avg_with.keys():
            diff = avg_with[metric] - avg_without[metric]
            print(f"  {metric}: {diff:.4f} ({'improved' if diff > 0 else 'decreased'})")

        if self.wandb_run is not None:
            self.wandb_run.log({
                **{f'std_{k}': v for k, v in avg_without.items()},
                **{f'over_{k}': v for k, v in avg_with.items()},
                **{f'diff_{k}': avg_with[k] - avg_without[k] for k in avg_with.keys()}
            })

    return {
        'without_oversampling': avg_without if results_without_oversampling else {},
        'with_oversampling': avg_with if results_with_oversampling else {}
    }

class CustomSAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super().__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.lin_self = nn.Linear(in_channels, out_channels)
        self.lin_neigh = nn.Linear(in_channels, out_channels)

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin_self.weight)
        nn.init.xavier_uniform_(self.lin_neigh.weight)

    def forward(self, x, edge_index, edge_weight=None):
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight)
        out = self.lin_neigh(out)
        out = out + self.lin_self(x)

        return out

    def message(self, x_j, edge_weight=None):
        if edge_weight is not None:
            return x_j * edge_weight.view(-1, 1)
        return x_j


def preprocess_with_smote(df_features, df_edges, oversample_ratio=30.0):
    print("\n=== Preprocessing with GraphSMOTE ===")
    features_df = df_features.copy()
    edges_df = df_edges.copy()

    max_node_id = features_df['node_id'].max()
    if isinstance(max_node_id, str):
        max_node_id = max([int(nid.split('_')[-1]) if '_' in nid else 0
                          for nid in features_df['node_id']])
    next_node_id = max_node_id + 1
    new_nodes = []
    new_edges = []
    all_timesteps = sorted(features_df['Time step'].unique())

    for timestep in all_timesteps:
        timestep_features = features_df[features_df['Time step'] == timestep]

        if 'binary_class' in timestep_features.columns:
            minority_nodes = timestep_features[timestep_features['binary_class'] == 1]
            majority_nodes = timestep_features[timestep_features['binary_class'] == 0]
        else:
            minority_nodes = timestep_features[timestep_features['class'] == 1]
            majority_nodes = timestep_features[timestep_features['class'] == 2]

        if len(minority_nodes) == 0 or len(minority_nodes) >= len(majority_nodes):
            print(f"Timestep {timestep}: Skipping (minority: {len(minority_nodes)}, majority: {len(majority_nodes)})")
            continue

        target_samples = min(int(len(minority_nodes) * oversample_ratio), len(majority_nodes))
        samples_to_generate = max(0, target_samples - len(minority_nodes))

        print(f"Timestep {timestep}: Oversampling minority class. Original: {len(minority_nodes)}, Target: {target_samples}")

        if samples_to_generate == 0:
            continue

        node_to_idx = {node: idx for idx, node in enumerate(timestep_features['node_id'])}
        idx_to_node = {idx: node for node, idx in node_to_idx.items()}

        feature_cols = timestep_features.columns.difference(['node_id', 'Time step', 'class', 'binary_class'])
        X = torch.tensor(timestep_features[feature_cols].values, dtype=torch.float32)

        if 'binary_class' in timestep_features.columns:
            y = torch.tensor(timestep_features['binary_class'].values, dtype=torch.long)
        else:
            y = torch.tensor(timestep_features['class'].map(lambda x: 1 if x == 1 else (0 if x == 2 else -1)).values,
                             dtype=torch.long)

        timestep_edges = edges_df[
            edges_df['source'].isin(timestep_features['node_id']) &
            edges_df['target'].isin(timestep_features['node_id'])
        ]

        edge_index = []
        for _, edge in timestep_edges.iterrows():
            if edge['source'] in node_to_idx and edge['target'] in node_to_idx:
                edge_index.append([node_to_idx[edge['source']], node_to_idx[edge['target']]])

        edge_index = torch.tensor(edge_index, dtype=torch.long).t() if edge_index else torch.zeros((2, 0), dtype=torch.long)

        try:
            new_X, new_edge_index, new_y = graph_smote(
                X, edge_index, y,
                minority_class=1,
                k_neighbors=5,
                n_synthetic=samples_to_generate
            )

            num_original_nodes = X.size(0)
            num_synthetic_nodes = new_X.size(0) - num_original_nodes

            if num_synthetic_nodes > 0:
                print(f"  Created {num_synthetic_nodes} synthetic nodes.")

                for i in range(num_original_nodes, new_X.size(0)):

                    if isinstance(max_node_id, str):
                        new_id = f"synthetic_{next_node_id}"
                    else:
                        new_id = f"synthetic_{next_node_id}"
                    next_node_id += 1

                    node_data = {
                        'node_id': new_id,
                        'Time step': timestep,
                        'binary_class': new_y[i].item(),
                        'class': 1 if new_y[i].item() == 1 else 2,
                        **{col: new_X[i, j].item() for j, col in enumerate(feature_cols)}
                    }
                    new_nodes.append(node_data)

                    idx_to_node[i] = new_id

                for j in range(new_edge_index.size(1)):
                    src_idx, dst_idx = new_edge_index[0, j].item(), new_edge_index[1, j].item()
                    if src_idx >= num_original_nodes or dst_idx >= num_original_nodes:
                        if src_idx in idx_to_node and dst_idx in idx_to_node:
                            new_edges.append({
                                'source': idx_to_node[src_idx],
                                'target': idx_to_node[dst_idx]
                            })

        except Exception as e:
            print(f"Error in GraphSMOTE for timestep {timestep}: {e}")
            continue

    if new_nodes:
        synthetic_nodes_df = pd.DataFrame(new_nodes)
        oversampled_features = pd.concat([features_df, synthetic_nodes_df], ignore_index=True)
        print(f"Added {len(synthetic_nodes_df)} synthetic nodes to features DataFrame.")
    else:
        oversampled_features = features_df
        print("No synthetic nodes were added.")

    if new_edges:
        synthetic_edges_df = pd.DataFrame(new_edges)
        oversampled_edges = pd.concat([edges_df, synthetic_edges_df], ignore_index=True)
        print(f"Added {len(synthetic_edges_df)} synthetic edges to edges DataFrame.")
    else:
        oversampled_edges = edges_df
        print("No synthetic edges were added.")

    if 'binary_class' in oversampled_features.columns:
        print("\nClass distribution after oversampling:")
        print(oversampled_features['binary_class'].value_counts().sort_index())
    else:
        print("\nClass distribution after oversampling:")
        print(oversampled_features['class'].value_counts().sort_index())

    return oversampled_features, oversampled_edges




In [ ]:
import time

trainer = EnhancedModelTrainer(
    checkpoint_dir=Path(save_path) / 'processed_data',
    wandb_project='Sage_',
    wandb_entity='ziprikk-university-duisburg',
    wandb_run=wandb.init(
        project='Sage_',
        entity='ziprikk-university-duisburg',
        config={
            "optimizer": "Adam",
            "architecture": "AdvancedGraphSAGE",
            "balance_strategy": "class_weights+focal_loss",
            "evaluation": "cross_validation"
        }
    )
)


Using device: cuda:0
Initial GPU memory allocated: 0.00 MB
Initial GPU memory cached: 0.00 MB

=== Preprocessing with GraphSMOTE (Timesteps 1-36) ===
Will only process timesteps: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Timestep 1: Oversampling minority class. Original: 96, Target: 4800
Debug - x shape: torch.Size([51108, 55]), num synthetic features: 1801
First synthetic feature shape: torch.Size([1, 55])
Success - new_x shape: torch.Size([52909, 55])
  Created 1801 synthetic nodes.
Timestep 2: Oversampling minority class. Original: 99, Target: 4950
Debug - x shape: torch.Size([32269, 55]), num synthetic features: 1435
First synthetic feature shape: torch.Size([1, 55])
Success - new_x shape: torch.Size([33704, 55])
  Created 1435 synthetic nodes.
Timestep 3: Oversampling minority class. Original: 58, Target: 2900
Debug - x shape: torch.Size([31106, 55]), num synthetic features: 862
First sy

In [ ]:

#trainer.debug_mask_creation()
# Run parameter search
#print("Starting hyperparameter optimization...")
#best_params = trainer.run_parameter_search(n_trials=50)
#print(f"Best parameters found: {best_params}")


In [ ]:
best_params = {
    'num_layers': 1,
    'base_channels': 240,
    'dropout': 0.43531998658753374,
    'learning_rate': 4.890518499140206e-05,
    'weight_decay': 0.0015759308553133077,
    'residual': False,
    'activation': 'leaky_relu',
    'edge_dropout': 0.01765443283938786,
    'focal_gamma': 2.7905145019036874,
    'class_weight_factor': 1.0296382120174215,
    'max_neighbors': 7,
    'layer_0_channels': 202,
    'aggregation_layer_0': 'sum',
    'neighbors_layer_0': 5
}

In [ ]:
#print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Train the final model
print("\nTraining final model with oversampling...")
model, metrics = trainer.train_final_model_with_timesteps(best_params)

GPU device: NVIDIA L4
GPU memory: 23.80 GB

Training final model with oversampling...

Training final model with best parameters...
Using timesteps [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36] for training
Using timesteps [37, 38, 39, 40, 41] for validation/threshold optimization
Using timesteps [42, 43, 44, 45, 46, 47, 48, 49] for final evaluation

Training with hyperparameters:
  Learning rate: 4.890518499140206e-05
  Weight decay: 0.0031518617106266154
  Edge dropout: 0.026481649259081787
  Focal gamma: 2.7905145019036874
  Class weight factor: 1.0296382120174215
  Original class weights: tensor([ 1.0844, 12.8482], device='cuda:0')
  Scaled class weights: tensor([ 1.0844, 13.2290], device='cuda:0')
Epoch 0: Avg Train Loss = 2.7894, Val Score = 0.4871
  class_1_precision: 0.1856
  class_1_recall: 0.6937
  class_1_f1: 0.2827
  composite_score: 0.4871
Epoch 5: Avg Train Loss = 1.1755, Val Score =

In [ ]:
# Extract embeddings for all timesteps and save to disk
embeddings_output_path = os.path.join(save_path, "node_embeddings.parquet")
embeddings_df = trainer.extract_node_embeddings(
    model=model,
    timesteps=None,
    output_path=embeddings_output_path
)

print(f"Node embeddings saved to: {embeddings_output_path}")

Using num_neighbors: [15]
Model structure:
AdvancedGraphSAGE(
  (layers): ModuleList(
    (0): CustomSAGEConv(55, 202)
  )
  (linear): Linear(in_features=202, out_features=2, bias=True)
  (dropouts): ModuleList(
    (0): Dropout(p=0.43531998658753374, inplace=False)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm1d(202, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

Model modules:
  : AdvancedGraphSAGE
  layers: ModuleList
  layers.0: CustomSAGEConv
  layers.0.aggr_module: SumAggregation
  layers.0.lin_self: Linear
  layers.0.lin_neigh: Linear
  linear: Linear
  dropouts: ModuleList
  dropouts.0: Dropout
  batch_norms: ModuleList
  batch_norms.0: BatchNorm1d
Processing timestep 1
Found final classification layer: linear
Using pre-hook on classification layer to capture embeddings
Captured embeddings shape: torch.Size([52909, 202])
Processing timestep 2
Found final classification layer: linear
Using pre-hook on classification layer to capture embeddings
Captu